In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

filename = '../input/global-peace-index/peace_index.csv'
df = pd.read_csv(filepath_or_buffer=filename, delimiter=';')
df.head()

,Country,iso3c,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,AFG,"3,153","3,246","3,087","3,038","3,069","3,101","3,095","3,108","3,369","3,48","3,513","3,636","3,626","3,626","3,626","3,434"
1,Angola,AGO,"1,655","1,827","1,615","1,816","1,615","1,615","1,609","1,408","1,403","1,403","1,61","1,615","1,413","1,621","1,608","1,639"
2,Albania,ALB,"1,247","1,27","1,293","1,293","1,293","1,282","1,27","1,259","1,259","1,247","1,236","1,224","1,414","1,403","1,403","1,403"
3,United Arab Emirates,ARE,1,1,1,"1,213","1,238","1,241","1,253","1,264","1,477","1,517","1,563","1,598","1,833","1,701","1,689","1,678"
4,Argentina,ARG,"1,403","1,604","1,604","1,403","1,403","1,604","1,403","1,403","1,403","1,403","1,201","1,201","1,201","1,201","1,201","1,201"
